In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI 蒸馏模型

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/distillation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在 Colab 中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/distillation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在 GitHub 上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/generative_ai/distillation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在 Vertex AI Workbench 中打开
    </a>
  </td>                                                                                               
</table>

注意：此笔记本已在以下环境中进行测试：

* Python 版本 = 3.9

## 概述

本教程演示了如何在Vertex AI上逐步使用蒸馏步骤。

我们开发了逐步蒸馏（DSS）方法（[论文](https://arxiv.org/abs/2305.02301v1)），可以通过从大型语言模型（LLM）中引出推理过程（基本原理）来丰富客户数据。这种新机制已被证明能够（a）训练出性能优于LLMs的更小模型，并且（b）通过利用较少的训练数据来实现这一点，而不是需要微调或蒸馏。我们的方法在多任务训练框架中提取LLM的基本原理作为额外的监督。

了解更多关于[蒸馏文本模型](https://cloud.google.com/vertex-ai/docs/generative-ai/models/distill-text-models)。

### 目标

在本教程中，您将学习如何使用 `Vertex AI LLM` 来提炼和部署一个大型语言模型。

本教程使用以下谷歌云ML服务：

- `Vertex AI LLM`
- `Vertex AI Model Garden`
- `Vertex AI Prediction`

执行的步骤包括：

- 获取 Vertex AI LLM 模型。
- 提炼模型。
  - 这将自动创建一个Vertex AI端点并将模型部署到其中。
- 使用 `Vertex AI LLM` 进行预测。
- 使用 `Vertex AI Prediction` 进行预测。

### 费用

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)、[云存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成费用估算。

## 安装

安装以下所需的软件包以执行这个笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform "shapely<2.0.0"

### 仅限Colab：取消注释下面的单元格以重新启动内核。

In [ ]:
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，都需要执行以下步骤。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得300美元的免费信用额度，可用于计算/存储成本。

2. [确保为您的项目启用了结算](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

4. 如果您在本地运行本笔记本，则需要安装[Cloud SDK](https://cloud.google.com/sdk)。

#### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参阅支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的谷歌云账户

根据您的Jupyter环境，您可能需要手动进行身份验证。请按照下面的相关说明操作。

1. 顶点 AI 工作台
* 无需操作，因为您已经通过验证。

2. 本地JupyterLab实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账户或其他
* 请参考如何为您的服务账户授予云存储权限，网址为https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

创建一个云存储桶

创建一个存储桶来存储中间产物，例如数据集。

- *{给笔记本作者的提示：对于任何需要是唯一的用户提供的字符串（例如存储桶名称或模型ID），请在末尾添加“-unique”以便进行适当测试}*

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶不存在的情况下才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} -p {PROJECT_ID} {BUCKET_URI}

数据集

我们已经为您提供了以下示例数据，供您开始使用。

In [ ]:
! gsutil cp gs://cloud-samples-data/vertex-ai/model-evaluation/peft_eval_sample.jsonl {BUCKET_URI}/peft_eval_sample.jsonl
! gsutil cp gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl {BUCKET_URI}/peft_train_sample.jsonl

#### 数据输入格式要求

蒸馏可以在带标签或不带标签的数据集上进行。如果您有一个高质量的带标签数据集，拥有数百个例子，我们建议您使用该数据集。否则，您可以使用一个无标签提示数据集。如果您使用无标签数据集，则教师模型会生成标签和蒸馏的理由。如果使用无标签数据集，则建议使用超过1,000个例子。

带标签或无标签的蒸馏数据集必须采用JSON行（JSONL）格式，其中每一行包含一个单独的调整示例。在蒸馏模型之前，您需要将数据集上传到云存储桶中。

每个数据集示例包含一个`input_text`字段，其中包含模型提示，以及一个可选的`output_text`字段，其中包含蒸馏模型预期产生的示例响应。

`input_text`的最大标记长度为7,168，`output_text`的最大标记长度为1,024。如果任一字段超过最大标记长度，则将截断多余标记。

一个文本生成模型的数据集中包含的示例最大数量为10,000。

示例数据集：

```
{"input_text": "question: How many people live in Beijing? context: With over 21 million residents, Beijing is the world's most populous national capital city and is China's second largest city after Shanghai. It is located in Northern China, and is governed as a municipality under the direct administration of the State Council with 16 urban, suburban, and rural districts.[14] Beijing is mostly surrounded by Hebei Province with the exception of neighboring Tianjin to the southeast; together, the three divisions form the Jingjinji megalopolis and the national capital region of China.", "output_text": "over 21 million people"}
{"input_text": "question: How many parishes are there in Louisiana? context: The U.S. state of Louisiana is divided into 64 parishes (French: paroisses) in the same manner that 48 other states of the United States are divided into counties, and Alaska is divided into boroughs.", "output_text": "64"}
```

### 导入库

In [ ]:
import vertexai
from google.cloud import aiplatform
from vertexai.preview.language_models import (TextGenerationModel,
                                              TuningEvaluationSpec)

### 初始化 Python 的 Vertex AI SDK

为您的项目初始化 Python 的 Vertex AI SDK。

In [ ]:
vertexai.init(project=PROJECT_ID, location=REGION)

加载预训练模型

从Vertex AI LLM Model Garden加载预训练的BISON模型。
查看支持蒸馏的模型[此处](https://cloud.google.com/vertex-ai/docs/generative-ai/models/distill-text-models#supported_models)。

In [ ]:
student_model = TextGenerationModel.from_pretrained("text-bison@002")
teacher_model = TextGenerationModel.from_pretrained(
    "text-unicorn@001"
)  # you can also use string 'text-unicorn@001'

### 提取模型

接下来，使用`distill_from()`方法来提取模型，使用以下参数：

`teacher_model`：您想要从中提取知识的教师模型。
`dataset`：用于调整模型的训练数据的pandas数据帧或云存储位置。
`learning_rate_multiplier`：应用于建议学习率的乘数。要使用建议的学习率，请使用1.0。
`train_steps`：用于模型调整的运行步数。批量大小根据调整位置而变：<br>
- us-central1批量大小为8。
- europe-west4批量大小为24。<br>

如果训练数据集中有240个示例，在europe-west4中，需要240 / 24 = 10个步骤来处理整个数据集一次。在us-central1中，需要240 / 8 = 30个步骤来处理整个数据集一次。默认值为300。<br>

要了解更多上下文，请参阅此 [文档](https://cloud.google.com/vertex-ai/docs/generative-ai/models/distill-text-models#create_a_text_model_distilling_job) 以了解参数的定义。

In [ ]:
# Optional: TuningEvaluationSpec
# see https://cloud.google.com/vertex-ai/docs/generative-ai/models/distill-text-models#create_a_text_model_distilling_job for full context

eval_spec = TuningEvaluationSpec()
eval_spec.evaluation_data = f"{BUCKET_URI}/peft_eval_sample.jsonl"
eval_spec.evaluation_interval = 20

In [ ]:
student_model.distill_from(
    teacher_model=teacher_model,
    dataset=f"{BUCKET_URI}/peft_train_sample.jsonl",
    train_steps=200,
    learning_rate_multiplier=1,
    accelerator_type="TPU",
    model_display_name="test-vertex-distillation",
    evaluation_spec=eval_spec,
)

使用 Vertex AI LLM 接口中的 `predict()` 方法进行预测。

In [ ]:
prompt = "TRANSCRIPT: \nPROCEDURE PERFORMED: , Umbilical hernia repair.,PROCEDURE:,  After informed consent was obtained, the patient was brought to the operative suite and placed supine on the operating table.  The patient was sedated, and an adequate local anesthetic was administered using 1% lidocaine without epinephrine.  The patient was prepped and draped in the usual sterile manner.,A standard curvilinear umbilical incision was made, and dissection was carried down to the hernia sac using a combination of Metzenbaum scissors and Bovie electrocautery.  The sac was cleared of overlying adherent tissue, and the fascial defect was delineated.  The fascia was cleared of any adherent tissue for a distance of 1.5 cm from the defect.  The sac was then placed into the abdominal cavity and the defect was closed primarily using simple interrupted 0 Vicryl sutures.  The umbilicus was then re-formed using 4-0 Vicryl to tack the umbilical skin to the fascia.,The wound was then irrigated using sterile saline, and hemostasis was obtained using Bovie electrocautery.  The skin was approximated with 4-0 Vicryl in a subcuticular fashion.  The skin was prepped with benzoin, and Steri-Strips were applied.  A dressing was then applied.  All surgical counts were reported as correct.,Having tolerated the procedure well, the patient was subsequently taken to the recovery room in good and stable condition.\n\n LABEL: "

In [ ]:
print(student_model.predict(prompt))

## 清理

要清理此项目中使用的所有Google Cloud资源，您可以[删除您用于教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源。

In [ ]:
import os

delete_bucket = False

if not os.getenv("IS_TESTING"):
    # Delete endpoint resource
    endpoint = aiplatform.Endpoint(student_model._endpoint.resource_name)
    endpoint.undeploy_all()
    endpoint.delete()

In [ ]:
delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}